<a href="https://colab.research.google.com/github/RyutoYoda/home_predict_proto/blob/main/home_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/home')

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [26]:
# データの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [45]:
train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
0,0,0,0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878,...,0.0,0.0,0.0,-292.0,0.0,0.0,0.0,0.0,1,0
1,1,0,0,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,365243,...,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0,1,0
2,2,0,0,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523,...,0.0,4.0,0.0,-542.0,0.0,0.0,0.0,0.0,1,0
3,3,0,0,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,365243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,4,1,0,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697,...,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0,1,0


In [44]:
test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
0,171202,1,144000.0,961146.0,28233.0,688500.0,0.025164,-12108,-2372,-2446.0,...,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,1,0
1,171203,0,103500.0,296280.0,16069.5,225000.0,0.007020,-17907,-1712,-10450.0,...,0.0,5.0,0.0,-212.0,0.0,0.0,0.0,0.0,1,0
2,171204,1,180000.0,183694.5,11236.5,139500.0,0.006852,-15221,-553,-1056.0,...,0.0,7.0,0.0,-428.0,0.0,1.0,1.0,1.0,1,0
3,171205,2,225000.0,450000.0,22500.0,450000.0,0.035792,-11217,-1438,-6096.0,...,0.0,2.0,0.0,-442.0,0.0,0.0,0.0,3.0,0,1
4,171206,2,144000.0,545040.0,26640.0,450000.0,0.020713,-11415,-2362,-3257.0,...,0.0,2.0,0.0,-1333.0,0.0,0.0,0.0,3.0,1,0


In [29]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [32]:
train = pd.get_dummies(train, columns=['NAME_CONTRACT_TYPE'])
test = pd.get_dummies(test, columns=['NAME_CONTRACT_TYPE'])

In [43]:
train = train.drop(['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','ORGANIZATION_TYPE'], axis=1)

In [40]:
test = test.drop(['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','ORGANIZATION_TYPE'], axis=1)

In [46]:
X = train.drop('TARGET', axis=1)
y = train['TARGET']

In [47]:
# 訓練データと検証データに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# LightGBMのデータセットに変換
train_dataset = lgb.Dataset(X_train, label=y_train)
val_dataset = lgb.Dataset(X_val, label=y_val)

In [49]:
# LightGBMのハイパーパラメータの設定
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 63,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'random_state': 42
}

In [50]:
# LightGBMモデルの学習
model = lgb.train(params, train_dataset, valid_sets=[train_dataset, val_dataset], early_stopping_rounds=100, verbose_eval=20)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training until validation scores don't improve for 100 rounds
[20]	training's binary_logloss: 0.254151	valid_1's binary_logloss: 0.26156
[40]	training's binary_logloss: 0.244568	valid_1's binary_logloss: 0.255943
[60]	training's binary_logloss: 0.238615	valid_1's binary_logloss: 0.253954
[80]	training's binary_logloss: 0.233576	valid_1's binary_logloss: 0.252889
[100]	training's binary_logloss: 0.229176	valid_1's binary_logloss: 0.25247
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.229176	valid_1's binary_logloss: 0.25247


In [51]:
# テストデータに対して予測を行う
predictions = model.predict(test)

In [54]:
print(predictions)

[0.03263082 0.1845151  0.18840129 ... 0.09425011 0.14386511 0.15434238]


In [52]:
# 予測結果をDataFrameに変換
submit_df = pd.DataFrame({'SK_ID_CURR': test['SK_ID_CURR'], 'TARGET': predictions})

# CSVファイルとして保存
submit_df.to_csv('/content/drive/MyDrive/home/submit.csv', index=False)

In [53]:
# 予測結果をDataFrameに変換
submit_df = pd.DataFrame({'SK_ID_CURR': test['SK_ID_CURR'], 'TARGET': predictions})

# 小数点以下1桁に変換
submit_df['TARGET'] = submit_df['TARGET'].round(1)

# CSVファイルとして保存
submit_df.to_csv('/content/drive/MyDrive/home/submit.csv', index=False)